## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
df = pd.read_csv('/kaggle/input/online-retail-dataset/online_retail.csv')
print(df.shape)
df.head()

(541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [49]:
# Function to view basic properties of the data
def display_profile(data):
    shape = data.shape[0]
    types = data.dtypes
    count = data.apply(lambda x: x.count())
    nulls = data.apply(lambda x: x.isnull().sum())
    distincts = data.apply(lambda x: x.unique().shape[0])
    missing_ratio = (data.isnull().sum())/shape*100
    skewness = data.skew(numeric_only = True)
    kurtosis = data.kurt(numeric_only=True)
    std = data.std(numeric_only=True)
    numeric_cols = data.select_dtypes(include=['number']).columns

    minimum = data.min(numeric_only=True)# data.apply(lambda x: x.min(numeric_only=True))
    maximum = data.max(numeric_only=True)# data.apply(lambda x: x.max(numeric_only=True))
    q1 = data.quantile(0.25,numeric_only=True)
    q2 = data.quantile(0.5,numeric_only=True)
    q3 = data.quantile(0.75,numeric_only=True)
    cols = ['types','count','nulls','missing_%','distincts','skewness','kurtosis','min','max','25%','50%','75%','std']
    df = pd.concat([types,count,nulls,missing_ratio,distincts,skewness,kurtosis,minimum,maximum,q1,q2,q3,std],axis = 1)
    df.columns = cols
    display(df)
    return df

In [50]:
profile1 = display_profile(df)

,types,count,nulls,missing_%,distincts,skewness,kurtosis,min,max,25%,50%,75%,std
InvoiceNo,object,397884,0,0.0,18532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
StockCode,object,397884,0,0.0,3665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Description,object,397884,0,0.0,3877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Quantity,int64,397884,0,0.0,301,409.892972,178186.243253,1.000,80995.00,2.00,6.00,12.00,179.331775
InvoiceDate,object,397884,0,0.0,17282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UnitPrice,float64,397884,0,0.0,440,204.032727,58140.396673,0.001,8142.75,1.25,1.95,3.75,22.097877
CustomerID,object,397884,0,0.0,4338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Country,object,397884,0,0.0,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Therefore, there are certain transactions with **negative quantity** and **negative unitprice**. There are some transactions with **no customer ids**. CustomerID is having float value,we need to make it object type.
Checking out these....

In [37]:
print(f"No.of transactions with non-positive quantity:{df[df.Quantity<=0].shape[0]}")
print(f"Percentage of transactions with non-positive quantity:{df[df.Quantity<=0].shape[0]/df.shape[0]*100:.2f}%")

No.of transactions with non-positive quantity:10624
Percentage of transactions with non-positive quantity:1.96%


In [38]:
print(f"No.of transactions with non-positive UnitPrice:{df[df.UnitPrice<=0].shape[0]}")
print(f"Percentage of transactions with non-positive UnitPrice:{df[df.UnitPrice<=0].shape[0]/df.shape[0]*100:.2f}%")

No.of transactions with non-positive UnitPrice:2517
Percentage of transactions with non-positive UnitPrice:0.46%


In [39]:
df = df[df.Quantity>0]
df = df[df.UnitPrice>0]
df = df[~df.CustomerID.isnull()]
df['CustomerID']= df['CustomerID'].astype('str')
df.shape[0]

397884

In [51]:
profile2 = display_profile(df)

,types,count,nulls,missing_%,distincts,skewness,kurtosis,min,max,25%,50%,75%,std
InvoiceNo,object,397884,0,0.0,18532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
StockCode,object,397884,0,0.0,3665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Description,object,397884,0,0.0,3877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Quantity,int64,397884,0,0.0,301,409.892972,178186.243253,1.000,80995.00,2.00,6.00,12.00,179.331775
InvoiceDate,object,397884,0,0.0,17282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UnitPrice,float64,397884,0,0.0,440,204.032727,58140.396673,0.001,8142.75,1.25,1.95,3.75,22.097877
CustomerID,object,397884,0,0.0,4338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Country,object,397884,0,0.0,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are 3665 distinct StockCodes, so accordingly there should also 3665 respective product descriptions. But there are 3877 descriptions...thus inconsistency. Some StockCodes might be having different descriptions, which isn't good.

In [52]:
codes = df.groupby(['StockCode','Description']).count().reset_index()

In [53]:
codes.StockCode.value_counts()[codes.StockCode.value_counts()>1]

StockCode
23196    4
23236    4
23126    3
23396    3
23535    3
        ..
23536    2
22950    2
22949    2
23091    2
82486    2
Name: count, Length: 213, dtype: int64

In [54]:
unique_desc = df[["StockCode", "Description"]].groupby(by=["StockCode"]).\
                apply(pd.DataFrame.mode).reset_index(drop=True)

/tmp/ipykernel_33/4049684832.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  apply(pd.DataFrame.mode).reset_index(drop=True)


In [91]:
print("Here is the list of all stockcodes in transactions with their description")
unique_desc

Here is the list of all stockcodes in transactions with their description


,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE
...,...,...
3661,C2,CARRIAGE
3662,DOT,DOTCOM POSTAGE
3663,M,Manual
3664,PADS,PADS TO MATCH ALL CUSHIONS


In [56]:
df_1 = pd.merge(df,unique_desc,on = 'StockCode',how = 'inner')
df_1 = df_1.drop(['Description_x'],axis =1)
df_1=df_1.rename(columns={'Description_y':'Description'})

In [57]:
df_1.InvoiceDate = pd.to_datetime(df_1.InvoiceDate)
df_1['Amount'] = df_1.Quantity*df_1.UnitPrice

In [58]:
profile3 = display_profile(df_1)

,types,count,nulls,missing_%,distincts,skewness,kurtosis,min,max,25%,50%,75%,std
InvoiceNo,object,397884,0,0.0,18532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
StockCode,object,397884,0,0.0,3665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Quantity,int64,397884,0,0.0,301,409.892972,178186.243253,1.000,80995.00,2.00,6.00,12.00,179.331775
InvoiceDate,datetime64[ns],397884,0,0.0,17282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UnitPrice,float64,397884,0,0.0,440,204.032727,58140.396673,0.001,8142.75,1.25,1.95,3.75,22.097877
CustomerID,object,397884,0,0.0,4338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Country,object,397884,0,0.0,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Description,object,397884,0,0.0,3647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Amount,float64,397884,0,0.0,2939,451.443182,232155.117219,0.001,168469.60,4.68,11.80,19.80,309.071041


In [97]:
df_1.to_csv('refined-dataset.csv')

# Country Analysis

In [64]:
most_countries = df_1.groupby(by='Country').size().\
                 reset_index(name='count').sort_values(by='count',ascending=False).\
                 reset_index(drop=True)
country_sales = df_1.groupby(by='Country')['Amount'].sum().\
                reset_index(name='TotalAmount').sort_values(by='TotalAmount',ascending=False).\
                reset_index(drop=True)

In [74]:
top10_1 = most_countries.head(10)
top10_2 = country_sales.head(10)

In [75]:
percent_1 = top10_1['count'].sum()/most_countries['count'].sum()*100
percent_2 = top10_2['TotalAmount'].sum()/country_sales['TotalAmount'].sum()*100

In [76]:
barchart_1 = px.bar(data_frame = top10_1,x='Country',y='count',title = "Top 10 countries with most transactions")
barchart_2 = px.bar(data_frame = top10_2,x='Country',y='TotalAmount',title = "Top 10 countries with most sales amount")


In [77]:
# Combine both plots horizontally
customer_plot = make_subplots(rows=1, cols=2)

customer_plot.add_trace(barchart_1['data'][0], row=1, col=1)
customer_plot.add_trace(barchart_2['data'][0], row=1, col=2)

customer_plot.update_xaxes(title_text="Top 10 countries with most transactions<br> \
                                    Did {:.2f}% of transactions".format(percent_1), row=1, col=1)
customer_plot.update_yaxes(title_text="Number of transaction done", row=1, col=1)
customer_plot.update_xaxes(title_text="Top 10 countries with most sales amount <br> \
                                    Did {:.2f}% of sales amount".format(percent_2), row=1, col=2)
customer_plot.update_yaxes(title_text="Total amount spend", row=1, col=2)

customer_plot.update_layout(title='Country plot')

customer_plot.show()

In [85]:
uk = df[df['Country']=='United Kingdom'].shape[0]/df.shape[0]*100
print(f"Percentage of all transactions from UK is {uk:.2f}%")

Percentage of all transactions from UK is 89.05%


# Customers analysis

In [86]:
customer_count = df_1.groupby(by = 'CustomerID').size().\
                reset_index(name='count').sort_values(by='count',ascending=False).\
                reset_index(drop=True)
customer_amount = df_1.groupby(by = 'CustomerID')['Amount'].sum().\
                reset_index(name='TotalAmount').sort_values(by='TotalAmount',ascending=False).\
                reset_index(drop=True)

In [87]:
# Extracting top 20 customers by number of transactions
top20_1 = customer_count.head(20).sort_values(by='count',ascending=False)
top20_1['CustomerID'] = top20_1['CustomerID'].astype('str')

# Extracting top 20 customers by sales amount
top20_2 = customer_amount.head(20).sort_values(by='TotalAmount',ascending=False)
top20_2['CustomerID'] = top20_2['CustomerID'].astype('str')

In [88]:
# Calcualating percentage of transactions and sales amount done by top 20 customers
percent_1 = top20_1['count'].sum()/customer_count['count'].sum()*100
percent_2 = top20_2['TotalAmount'].sum()/customer_amount['TotalAmount'].sum()*100

### Plotting Customers

In [89]:
# Bar charts to plot the customers
barchart_1 = px.bar(data_frame = top20_1,x='CustomerID',y='count',title = "Top 20 customers by number of transactions")
barchart_2 = px.bar(data_frame = top20_2,x='CustomerID',y='TotalAmount',title = "Top 20 customers by sales amount")

In [90]:
# Combine both plots horizontally
customer_plot = make_subplots(rows=1, cols=2)

customer_plot.add_trace(barchart_1['data'][0], row=1, col=1)
customer_plot.add_trace(barchart_2['data'][0], row=1, col=2)

customer_plot.update_xaxes(title_text="Top 20 customers by number of transactions<br> \
                                    Did {:.2f}% of transactions".format(percent_1), row=1, col=1)
customer_plot.update_yaxes(title_text="Number of transaction done", row=1, col=1)
customer_plot.update_xaxes(title_text="Top 20 customers by sales amount <br> \
                                    Did {:.2f}% of sales amount".format(percent_2), row=1, col=2)
customer_plot.update_yaxes(title_text="Total amount spend", row=1, col=2)

customer_plot.update_layout(title='Customers plot')

customer_plot.show()


# Product Analysis

In [92]:
product_count = df_1.groupby(by = 'Description').size().\
                reset_index(name='count').sort_values(by='count',ascending=False).\
                reset_index(drop=True)
product_amount = df_1.groupby(by = 'Description')['Amount'].sum().\
                reset_index(name='TotalAmount').sort_values(by='TotalAmount',ascending=False).\
                reset_index(drop=True)

In [93]:
# Extracting top 10 products by number of transactions
top10_1 = product_count.head(20).sort_values(by='count',ascending=False)
top10_1['Description'] = top10_1['Description'].astype('str')

# Extracting top 10 products by sales amount
top10_2 = product_amount.head(20).sort_values(by='TotalAmount',ascending=False)
top10_2['Description'] = top10_2['Description'].astype('str')

In [94]:
# Calcualating percentage of transactions and sales amount done by top 20 customers
percent_1 = top10_1['count'].sum()/product_count['count'].sum()*100
percent_2 = top10_2['TotalAmount'].sum()/product_amount['TotalAmount'].sum()*100

In [95]:
# Bar charts to plot the customers
barchart_1 = px.bar(data_frame = top10_1,x='Description',y='count',title = "Top 10 products by number of transactions")
barchart_2 = px.bar(data_frame = top10_2,x='Description',y='TotalAmount',title = "Top 10 products by sales amount")

In [96]:
# Combine both plots horizontally
product_plot = make_subplots(rows=1, cols=2)

product_plot.add_trace(barchart_1['data'][0], row=1, col=1)
product_plot.add_trace(barchart_2['data'][0], row=1, col=2)

product_plot.update_xaxes(title_text="Top 10 products by number of transactions<br> \
                                    Did {:.2f}% of transactions".format(percent_1), row=1, col=1)
product_plot.update_yaxes(title_text="Number of transaction done", row=1, col=1)
product_plot.update_xaxes(title_text="Top 10 products by sales amount <br> \
                                    Did {:.2f}% of sales amount".format(percent_2), row=1, col=2)
product_plot.update_yaxes(title_text="Total amount spend", row=1, col=2)

product_plot.update_layout(title='Products plot')

product_plot.show()
